In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAEySagEAAAAARIqM1OXo7auNbvf3TVo1uu%2BcQRM%3DyNakXScQEIIMbF3RXLT1pvegwlwJyExjLJTNVFOZ0hCBRe3lGI'

In [3]:
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(keyword, start_date, end_date, max_results = 100):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions':  'author_id,in_reply_to_user_id,geo.place_id,attachments.media_keys,entities.mentions.username',
                    'tweet.fields': 'attachments,author_id,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'media.fields': 'media_key,preview_image_url,public_metrics,type,url',
                    'place.fields':'country,full_name,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [8]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    includes=json_response['includes']

    users=includes['users']
    users = {user["id"]: user for user in users}

    #medias = includes['media']
    #mediass = {media['media_key']: media for media in medias}
    media = {m["media_key"]: m for m in includes['media']}

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # Author 
        author_id = tweet['author_id']
        author_name=users[author_id]['name']

        #  Time created
        created_at = dateutil.parser.parse(tweet['created_at'])
        
        #attachments
        if('attachments' in tweet):
            attachment=tweet['attachments']
#             media_keys = attachment['media_keys']
        
#             media_key=media[media_keys[0]].media_key
#             media_url=media[media_keys[0]].url or media[media_keys[0]].preview_image_url
#             media_type=media[media_keys[0]].type
        else:
            attachment=" "
#             media_keys=" "

        # Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
#             name=geo['full_name']
        else:
            geo = " "

        # Tweet ID
        tweet_id = tweet['id']
        

        # Language
        lang = tweet['lang']

        # Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # source
        source = tweet['source']

        # Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
#         headers=['created at','id','author id','Author name','text','lang','like_count', 'quote_count', 
#                  'reply_count','retweet_count','attachments',
#                  'Media key','Media url','geo','Location','source']
        res = [created_at,tweet_id,author_id,author_name,text,lang,like_count, quote_count, reply_count,
               retweet_count,attachment,geo,source]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

In [ ]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
#hashtags related to 2022 general elections-AzimioLaUmoja, azimio, azimiostrong, UDAkwanza, 
#elections22, Elections2022,IEBC,kenyakwanza,
keyword = keyword = 'AzimioLaUmoja OR azimio OR azimiostrong OR UDAkwanza OR runningmate OR elections22 OR Elections2022 OR IEBC OR kenyakwanza -is:retweet has:media has:mentions has:geo lang:en (place_country:ke)'
#elections22, Elections2022,IEBC,kenyakwanza"
start_list =    [
                 '2022-01-01T00:00:00.000Z',
                 '2022-01-15T00:00:00.000Z',
                 '2022-02-01T00:00:00.000Z',
                 '2022-02-15T00:00:00.000Z',
                 '2022-03-01T00:00:00.000Z',
                 '2022-03-15T00:00:00.000Z',
                 '2022-04-01T00:00:00.000Z',
                 '2022-04-15T00:00:00.000Z',
                 '2022-05-01T00:00:00.000Z'
                
                ]

end_list =      [
                '2022-01-14T23:59:45.000Z',
                 '2022-01-31T00:00:00.000Z',
                 '2022-02-14T23:59:45.000Z',
                 '2022-02-28T00:00:00.000Z',
                '2022-03-14T23:59:45.000Z',
                 '2022-03-31T00:00:00.000Z',
                '2022-04-14T23:59:45.000Z',
                 '2022-04-30T00:00:00.000Z',
                 '2022-05-14T00:00:00.000Z'
                ]
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("AugustElections2022.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

csvWriter.writerow(['created at','id','author id','Author name','text','lang','like_count', 'quote_count', 'reply_count','retweet_count','attachments','geo','source'])
#csvWriter.writerow(['author id', 'created_at', 'geo','Location' 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
#csvWriter.writerow(headers)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
#csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 40000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "AugustElections2022.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(15)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45y9a8004teww1dum4r8db63mv1
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  482
Total # of Tweets added:  482
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45y9a8004teww1dum4r8db63mv1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45y99mdlp4n2deff20jpnc3qmil
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  479
Total # of Tweets added:  961
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45y99mdlp4n2deff20jpnc3qmil
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45y9911vbkdx6g6jb54clnatbp9
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  1438
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45y9911vbkdx6g6jb54clnatbp9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45y98q3do715vkars0fh9b7tf99
Sta

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45jggc9jk1t69566sazxbxshlrx
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  416
Total # of Tweets added:  13610
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45jggc9jk1t69566sazxbxshlrx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45jgg1k5utzriorepagxor85th9
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  422
Total # of Tweets added:  14032
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45jgg1k5utzriorepagxor85th9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45jgfqt7tg9wsu2bwsqgl5n8r5p
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  448
Total # of Tweets added:  14480
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45jgfqt7tg9wsu2bwsqgl5n8r5p
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45jgfg0t35g712jrhfzbw3lk41p
Start Date:  2022-01-01T00:00:0

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45j5s3y0cyjkhx8b18lqfc97mgt
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  494
Total # of Tweets added:  27092
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45j5s3y0cyjkhx8b18lqfc97mgt
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45j5rid5m38aaxub5p87etflunx
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  478
Total # of Tweets added:  27570
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45j5rid5m38aaxub5p87etflunx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45j5qwqsvtmb55dnpplcqpvcq65
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  28055
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe45j5qwqsvtmb55dnpplcqpvcq65
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe45j3nzgkm2ixtp9j1cu3ract8cd
Start Date:  2022-01-01T00:00:0

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe454aslheid7ljhmzz8dowdcr4ot
Start Date:  2022-01-01T00:00:00.000Z
# of Tweets added from this response:  464
Total # of Tweets added:  40454
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe482idquzwuj5vgnduuqfrf08nel
Start Date:  2022-01-15T00:00:00.000Z
# of Tweets added from this response:  492
Total # of Tweets added:  40946
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe482idquzwuj5vgnduuqfrf08nel
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe482id5d4pdsvstrassu2d3lhwn1
Start Date:  2022-01-15T00:00:00.000Z
# of Tweets added from this response:  493
Total # of Tweets added:  41439
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe482id5d4pdsvstrassu2d3lhwn1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe482icug5of8yv8j7wowgoav860t
Start Date:  2022-01-15T00:00:00.000Z
# of Tweets added from this respon

-------------------
Token:  b26v89c19zqg8o3fpe4829uufpaaz0qmofklb83am5bwd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe4829uu52x7k1utp84vtfvktu6il
Start Date:  2022-01-15T00:00:00.000Z
# of Tweets added from this response:  431
Total # of Tweets added:  53905
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe4829uu52x7k1utp84vtfvktu6il
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe4829ujiptngz9iavrzqacwwk3ul
Start Date:  2022-01-15T00:00:00.000Z
# of Tweets added from this response:  422
Total # of Tweets added:  54327
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe4829ujiptngz9iavrzqacwwk3ul
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe4829u8wcxrxxgvc0vpnotv6vywt
Start Date:  2022-01-15T00:00:00.000Z
# of Tweets added from this response:  423
Total # of Tweets added:  54750
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe4829u8wcxrxxgvc0vpnotv6vywt
Endpoint Response Code: 200
Next Token:  b26

...

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqm3aaxl47gxfuw7a44k6enmftdp
Start Date:  2022-03-01T00:00:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  200867
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqm3aaxl47gxfuw7a44k6enmftdp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqm3aaxdigfbzwmji4v2oel6cry5
Start Date:  2022-03-01T00:00:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  201343
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqm3aaxdigfbzwmji4v2oel6cry5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqm3aamsp9iok4i51e6i7udnawzh
Start Date:  2022-03-01T00:00:00.000Z
# of Tweets added from this response:  484
Total # of Tweets added:  201827
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqp151mi6r3a9ml3fm490qstae0t
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this res

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqomc755dvf4bdjfpv2oxsh861kt
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  456
Total # of Tweets added:  214137
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqomc755dvf4bdjfpv2oxsh861kt
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqomc6u5e0rcmd07ps3kbcz0mv7h
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  445
Total # of Tweets added:  214582
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqomc6u5e0rcmd07ps3kbcz0mv7h
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqomc6j6wdf1divkrgsf6ylogvp9
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  445
Total # of Tweets added:  215027
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqomc6j6wdf1divkrgsf6ylogvp9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqomc5xv4pvybf7md2ycf5v5e9od
Start Date:  2022-03-15T00:0

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqom1jid7va6h5qaf8dffo6rphtp
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  227301
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqom1jid7va6h5qaf8dffo6rphtp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqom1j7epd19rimihukwvanslqbh
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  454
Total # of Tweets added:  227755
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqom1j7epd19rimihukwvanslqbh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqom1iwg8karqatrz5depngw314t
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  422
Total # of Tweets added:  228177
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqom1iwg8karqatrz5depngw314t
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqom1ib5ygsy7jbngy2mrejsm73x
Start Date:  2022-03-15T00:0

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqo74hynhtos0n5iwkqgi93knddp
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  240539
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqo74hynhtos0n5iwkqgi93knddp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqo74hnzmb7wo6oxnulh69dpfl31
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  483
Total # of Tweets added:  241022
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqo74hnzmb7wo6oxnulh69dpfl31
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqo74hda8eclioc3xbkn9q0o1q4d
Start Date:  2022-03-15T00:00:00.000Z
# of Tweets added from this response:  487
Total # of Tweets added:  241509
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyqo74hda8eclioc3xbkn9q0o1q4d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpyqo74hd2lkpw3w4hosxditc034hp
Start Date:  2022-03-15T00:0

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytmpdauq3z2s08uv304ou8z6jou5
Start Date:  2022-04-01T00:00:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  253362
-------------------
-------------------
Token:  b26v89c19zqg8o3fpytmpdauq3z2s08uv304ou8z6jou5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytmpda9fvsmghadcz3p855cxv6v1
Start Date:  2022-04-01T00:00:00.000Z
# of Tweets added from this response:  467
Total # of Tweets added:  253829
-------------------
-------------------
Token:  b26v89c19zqg8o3fpytmpda9fvsmghadcz3p855cxv6v1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytmpd9yhcv5xc7rf0xqexawyvjp9
Start Date:  2022-04-01T00:00:00.000Z
# of Tweets added from this response:  473
Total # of Tweets added:  254302
-------------------
-------------------
Token:  b26v89c19zqg8o3fpytmpd9yhcv5xc7rf0xqexawyvjp9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpytmpd8rj9jj80gtf841a10vqowot
Start Date:  2022-04-01T00:0